In [3]:
from django.conf import settings
settings.configure(BASE_DIR='/Users/adammajczyk/Documents/AAA_Semy/Sem7/inzynierka/pracaInzynierska/djangoapp',)
print(settings.BASE_DIR)

RuntimeError: Settings already configured.

In [4]:
import sys
sys.path.append('/Users/adammajczyk/Documents/AAA_Semy/Sem7/inzynierka/pracaInzynierska')  # Replace with the actual path to the parent directory

from djangoapp.news.scripts.llm import *
from djangoapp.news.scripts.model_loader import *
from djangoapp.news.scripts.nlp import *

/Users/adammajczyk/Documents/AAA_Semy/Sem7/inzynierka/pracaInzynierska/djangoapp


In [3]:
model_loader = ModelLoader()

In [8]:
llm = model_loader.llm 
llm.predict('test')

0.00039659696631133556